In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/content/drive/MyDrive/campus x /Capstone Project /appartments.csv")

In [3]:
df.shape

(247, 7)

In [4]:
df.drop([22], inplace = True)

In [5]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


* PropertyName : Name of the apartment (we will need this column because we are
 going to give this column as a input to user)
* PropertySubName : This column contains information about number of rooms and sector
* NearbyLocations	 : This column contains information about the nearby location from a apartment
* LocationAdvantages : This column contains more near by locations (including NearbyLocations column) along with distances (we can say NearbyLocations column is the subset of this column)
* PriceDetails : This column contains detailed information about the property based on the number of rooms (i.e BHK)(we can say PropertySubName is a subset of this column )
* TopFacilities : Amenities present in particular apartment

Based on above information we will keep PropertyName (for user input)
and TopFacilities, PriceDetails, LocationAdvantages for simillarity finding and use for recommendation



We will be making 3 recommendetion engine 1st will be based on the simillarity of TopFacilities, 2nd will be based on the simillarity of pricedetails and 3rd will be based on the simillarity of locationadvantages

The reasion We are making three recommendetion engines seperatly is that we will be giving weightage to each 3 of recommendation engine and finally combin them all. By doing this we will have flexibility to increase the weightage of particular recommendation engine so that we will get more recommendation based on the simillarity of high weigtage recommendation engine




#Recommendetion Based On Top Facilities

In [6]:
df[['PropertyName', 'TopFacilities']]

,PropertyName,TopFacilities
0,Smartworld One DXP,"['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."
...,...,...
242,DLF Princeton Estate,"['Swimming Pool', 'Medical Centre', 'Laundry',..."
243,Pyramid Urban Homes 2,"['Shopping Centre', 'Community Hall', '24x7 Se..."
244,Satya The Hermitage,"['Bus Shelter', 'Swimming Pool', 'Business Lou..."
245,BPTP Spacio,"['Swimming Pool', 'Card Room', 'Piped Gas', 'P..."


In [7]:
#converting string to list
def lit_eval(obj):
  return re.findall(r"'(.*?)'", obj)
  #converting to list
df['TopFacilities'] = df['TopFacilities'].apply(lambda x: lit_eval(x))

In [8]:
#converting from list to single string
df['TopFacilities'] = df['TopFacilities'].apply(lambda x : ' '.join(x))

In [9]:
df['TopFacilities']

0      Swimming Pool Salon Restaurant Spa Cafeteria S...
1      Bowling Alley Mini Theatre Manicured Garden Sw...
2      Terrace Garden Gazebo Fountain Amphitheatre Pa...
3      Swimming Pool Volley Ball Court Aerobics Centr...
4      Mini Theatre Doctor on Call Concierge Service ...
                             ...                        
242    Swimming Pool Medical Centre Laundry Salon Gro...
243    Shopping Centre Community Hall 24x7 Security G...
244    Bus Shelter Swimming Pool Business Lounge Read...
245    Swimming Pool Card Room Piped Gas Pool Table S...
246    Swimming Pool School Solar Lighting Aerobics C...
Name: TopFacilities, Length: 246, dtype: object

* Now we will try to find the most simillar apartment based on TopFacilities.

* to find out the simillarirty we will convert each row in topfacilty into a vector form using tfidfvectorizer.

* Then we will find out the most nearest vector based on the angle(using cosine simillarity)
(we could have use Eucledian distance as well but in high dimensional space i.e when number of features are more then Euclidien distance does not give accurate value that is why we are using angular distance)

In [10]:
#reating object of Tfidfvectorizer
tfidf = TfidfVectorizer(stop_words = 'english', ngram_range = (1,2))

In [11]:
tfidf_matrix = tfidf.fit_transform(df['TopFacilities'])

In [12]:
tfidf_matrix.toarray()[0]

array([0.        , 0.        , 0.        , 0.18809342, 0.18809342,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [13]:
#lets find the angle of each vector with respect to each other vector using cosine_simillarity

cosine_sim_1= cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
cosine_sim_1.shape

(246, 246)

In [15]:
sorted(list(enumerate(cosine_sim_1[1])), key = lambda x: x[1], reverse = True)

[(1, 1.0000000000000002),
 (85, 0.35860320390686057),
 (226, 0.33176299517252417),
 (74, 0.3185862339669967),
 (145, 0.30917614904944274),
 (89, 0.29743060271126903),
 (67, 0.29741233419695945),
 (40, 0.28166250341736526),
 (82, 0.24264785543805475),
 (117, 0.2175472898913876),
 (197, 0.2174905090684951),
 (6, 0.21700920287542244),
 (20, 0.2162165818083719),
 (55, 0.2072417395227972),
 (127, 0.19584217615242325),
 (35, 0.19043481889738081),
 (122, 0.18545274846684248),
 (202, 0.18140142177520335),
 (25, 0.1731952137815659),
 (201, 0.16555034724610662),
 (80, 0.16301691925241446),
 (105, 0.15912758579528438),
 (4, 0.15888474057768506),
 (95, 0.1580492328467247),
 (183, 0.15716165955917108),
 (23, 0.15373791911523915),
 (231, 0.1532427959388649),
 (91, 0.15279624584655666),
 (78, 0.1517940606622079),
 (88, 0.15081343904630687),
 (27, 0.1499732693190523),
 (12, 0.1489600935113133),
 (164, 0.14579978851231856),
 (71, 0.1398717942223478),
 (98, 0.13804248157255858),
 (159, 0.131557857947159

In [16]:
def recommend_properties(property_name, cosin_sim = cosine_similarity):

  #get index
  idx = df[df['PropertyName'] == property_name].index[0]

  #get cosine simillarity of all apartment with respect to selected property_name
  cosine_list = list(enumerate(cosine_sim_1[idx]))

  #sort the lis based on cosine_similarity
  sorted_cosine_list = sorted(cosine_list, key = lambda x : x[1], reverse = True)

  #select top 6 higest cosine_similarity properties
  top_similar_property = sorted_cosine_list[:6]

  #get the index of most simillar properties
  top_idx = [tup[0] for tup in top_similar_property]

  recommendations_df = pd.DataFrame(
      {
          'property_name'  : df['PropertyName'].iloc[top_idx],
          'cosine_simillarity' : top_similar_property,
          'TopFacilities' : df['TopFacilities'].iloc[top_idx]
      }
  )

  return recommendations_df

In [17]:
recommend_properties('Smartworld One DXP')

,property_name,cosine_simillarity,TopFacilities
0,Smartworld One DXP,"(0, 1.0)",Swimming Pool Salon Restaurant Spa Cafeteria S...
11,DLF The Arbour,"(11, 0.388850461994329)",Swimming Pool Community Hall Yoga/Meditation A...
154,India Rashtra,"(153, 0.3459354538430299)",Swimming Pool CCTV Camera Security 24x7 Securi...
64,Ace Palm Floors,"(63, 0.3278166625200139)",Swimming Pool Fountain Sun Deck Amphitheatre Y...
114,M3M Natura,"(113, 0.31791768235666384)",Swimming Pool Football Bar/Chill-Out Lounge Sa...
116,Ireo City Plots,"(115, 0.30214002412270297)",Swimming Pool Salon Banquet Hall Lawn Tennis C...


#Recommendetion using PriceDetails

In [18]:
import json

In [19]:
df.iloc[[22]]['PriceDetails'].values

array(["{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,300 - 1,330 sq.ft.', 'price-range': '₹ 1.3 - 1.56 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,650 - 1,700 sq.ft.', 'price-range': '₹ 1.65 - 2.06 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,055 - 2,249 sq.ft.', 'price-range': '₹ 3.29 - 3.69 Cr'}}"],
      dtype=object)

To create a recommendation engine we need vectors.
SO to create vector from PriceDetails column we need to crate columns building_type, area and price-range for each BHK. Following code is all about create this columns

In [20]:
def refined_parse_modified (detail_str):

  #create empty dictionary which willl hold the all the parameters for all bhk ex area_2BHK, area_3 BHk etc
  extracted = {}
  #replace ' inside dictionary with " and then convert '{}' to {}
  details = json.loads(detail_str.replace("'", "\""))

  for bhk, det in details.items():  #detail.items() = will convert {2 BHK : {}} to bhk = 2BHK and det = {}

    # Extract building typ for each BHK in dictionary and store it in extracted
    try :
      extracted[f"building_type {bhk}"] = det['building_type']
    except:
      extracted[f"building_type {bhk}"] = None

    # Extract area for each BHK in dictionary and store it in extracted
    #use try and except so that if area is not given then pass None in except
    try:
      #split area to store low and high value in list
      area = det['area'].split("-")
      #if range is not given then use same area for both high and low
      if len(area) == 1:
        extracted[f"area_low {bhk}"] = float(area[0].replace(',', '').replace('sq.ft.', '').strip())
        extracted[f"area_high {bhk}"] = float(area[0].replace(',', '').replace('sq.ft.', '').strip())
      elif len(area) == 2:
        extracted[f"area_low {bhk}"] = float(area[0].replace(',', '').replace('sq.ft.', '').strip())
        extracted[f"area_high {bhk}"] = float(area[1].replace(',', '').replace('sq.ft.', '').strip())
      else :
        extracted[f"area_low {bhk}"] = None
        extracted[f"area_high {bhk}"] = None
    except :
        extracted[f"area_low {bhk}"] = None
        extracted[f"area_high {bhk}"] = None

    # Extract price-range for each BHK in dictionary and store it in extracted
     #use try and except so that if price-range is not given or if price-rang given is "Price on Demand" then pass None in except
    try:
      #split price-range to store low and high value in list
      price_range = det['price-range'].split("-")
      #if range is not given then use same price-range for both high and low
      if len(price_range) == 1:
        extracted[f"price_range_low {bhk}"] = float(price_range[0].replace("₹", '').replace('L', '').replace('Cr', ''))
        extracted[f"price_range_high {bhk}"] = float(price_range[0].replace("₹", '').replace('L', '').replace('Cr', ''))
        #if price is in lakh then div by 100
        if 'L' in price_range[0]:
          extracted[f"price_range_low {bhk}"] =  extracted[f"price_range_low {bhk}"]/100
          extracted[f"price_range_high {bhk}"] = extracted[f"price_range_high {bhk}"]/100

      elif len(price_range) == 2:
        extracted[f"price_range_low {bhk}"] = float(price_range[0].replace("₹", '').replace('L', '').replace('Cr', ''))
        extracted[f"price_range_high {bhk}"] = float(price_range[1].replace("₹", '').replace('L', '').replace('Cr', ''))
        if 'L' in price_range[0]:
          extracted[f"price_range_low {bhk}"] = extracted[f"price_range_low {bhk}"]/100
        if 'L' in price_range[1]:
          extracted[f"price_range_high {bhk}"] = extracted[f"price_range_high {bhk}"]/100
      else :
        extracted[f"price_range_low {bhk}"] = None
        extracted[f"price_range_high {bhk}"] = None
    except :
        extracted[f"price_range_low {bhk}"] = None
        extracted[f"price_range_high {bhk}"] = None

  return extracted

In [21]:
#Create list to create new dataframe which will consist the columns based on PriceDetails
list_for_df = []
#Iterate over all the apartments
for _, row in df.iterrows():
  #apply above function to get all the PriceDetails in properly segregated format
  feture = refined_parse_modified(row["PriceDetails"])

  #create dictonary new_row to create data frame
  new_row = {'PropertyName' : row["PropertyName"]}

  #iterate over all the possible BHK for each apartment so that we can create equal number of columns for each apartment and create data frame out of it.
  for bhk in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
    new_row[f"building_type_{bhk}"] = feture.get(f"building_type {bhk}") # we are using .get so if some bhk is absent then it will automatically pass nan value
    new_row[f"area_low_{bhk}"] = feture.get(f"area_low {bhk}")
    new_row[f"area_high_{bhk}"] = feture.get(f"area_high {bhk}")
    new_row[f"price_range_low_{bhk}"] = feture.get(f"price_range_low {bhk}")
    new_row[f"price_range_high_{bhk}"] = feture.get(f"price_range_high {bhk}")

  list_for_df.append(new_row)
  new_row = {}

In [22]:
pd.DataFrame(list_for_df)

,PropertyName,building_type_1 BHK,area_low_1 BHK,area_high_1 BHK,price_range_low_1 BHK,price_range_high_1 BHK,building_type_2 BHK,area_low_2 BHK,area_high_2 BHK,price_range_low_2 BHK,...,building_type_1 RK,area_low_1 RK,area_high_1 RK,price_range_low_1 RK,price_range_high_1 RK,building_type_Land,area_low_Land,area_high_Land,price_range_low_Land,price_range_high_Land
0,Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.0,1370.0,2.0000,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
1,M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
2,Adani Brahma Samsara Vilasa,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,,500.0,4329.0,2.05,41.13
3,Sobha City,None,NaN,NaN,NaN,NaN,Apartment,1381.0,1692.0,1.5500,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
4,Signature Global City 93,None,NaN,NaN,NaN,NaN,Independent Floor,981.0,1118.0,0.9301,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,DLF Princeton Estate,None,NaN,NaN,NaN,NaN,Apartment,964.0,964.0,1.3500,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
242,Pyramid Urban Homes 2,Apartment,335.0,398.0,23.45,0.2786,Apartment,500.0,625.0,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
243,Satya The Hermitage,None,NaN,NaN,NaN,NaN,Apartment,1450.0,1450.0,0.8000,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
244,BPTP Spacio,None,NaN,NaN,NaN,NaN,Apartment,1000.0,1079.0,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [23]:
#Create data frame
pd.DataFrame(list_for_df)
df_final_refined_v2 = pd.DataFrame(list_for_df).set_index('PropertyName')
df_final_refined_v2['building_type_Land'] = df_final_refined_v2['building_type_Land'].replace({'':'Land'})

In [24]:
df_final_refined_v2.shape

(246, 40)

In [25]:
#extract all categorical columns
categorical_columns = df_final_refined_v2.select_dtypes(include = 'object').columns.to_list()
categorical_columns

['building_type_1 BHK',
 'building_type_2 BHK',
 'building_type_3 BHK',
 'building_type_4 BHK',
 'building_type_5 BHK',
 'building_type_6 BHK',
 'building_type_1 RK',
 'building_type_Land']

In [26]:
#convert categorical columns to one hot encoded
ohe_df = pd.get_dummies(df_final_refined_v2, columns = categorical_columns, drop_first = True)
ohe_df.head(2)

,area_low_1 BHK,area_high_1 BHK,price_range_low_1 BHK,price_range_high_1 BHK,area_low_2 BHK,area_high_2 BHK,price_range_low_2 BHK,price_range_high_2 BHK,area_low_3 BHK,area_high_3 BHK,...,building_type_2 BHK_Independent Floor,building_type_2 BHK_Service Apartment,building_type_3 BHK_Independent Floor,building_type_3 BHK_Service Apartment,building_type_3 BHK_Villa,building_type_4 BHK_Independent Floor,building_type_4 BHK_Villa,building_type_5 BHK_Independent Floor,building_type_5 BHK_Villa,building_type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,NaN,NaN,NaN,NaN,1370.0,1370.0,2.0,2.4,1850.0,2050.0,...,0,0,0,0,0,0,0,0,0,0
M3M Crown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1605.0,2170.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#fill nan values with 0
ohe_df.fillna(0, inplace = True)
ohe_df.head(2)

,area_low_1 BHK,area_high_1 BHK,price_range_low_1 BHK,price_range_high_1 BHK,area_low_2 BHK,area_high_2 BHK,price_range_low_2 BHK,price_range_high_2 BHK,area_low_3 BHK,area_high_3 BHK,...,building_type_2 BHK_Independent Floor,building_type_2 BHK_Service Apartment,building_type_3 BHK_Independent Floor,building_type_3 BHK_Service Apartment,building_type_3 BHK_Villa,building_type_4 BHK_Independent Floor,building_type_4 BHK_Villa,building_type_5 BHK_Independent Floor,building_type_5 BHK_Villa,building_type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.0,0.0,1370.0,1370.0,2.0,2.4,1850.0,2050.0,...,0,0,0,0,0,0,0,0,0,0
M3M Crown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1605.0,2170.0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#lets standerdize the scale
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
ohe_df_normalized = pd.DataFrame(scale.fit_transform(ohe_df), columns = ohe_df.columns, index = ohe_df.index)

#finally we have created a vector from PriceDetails column
ohe_df_normalized.head(2)

,area_low_1 BHK,area_high_1 BHK,price_range_low_1 BHK,price_range_high_1 BHK,area_low_2 BHK,area_high_2 BHK,price_range_low_2 BHK,price_range_high_2 BHK,area_low_3 BHK,area_high_3 BHK,...,building_type_2 BHK_Independent Floor,building_type_2 BHK_Service Apartment,building_type_3 BHK_Independent Floor,building_type_3 BHK_Service Apartment,building_type_3 BHK_Villa,building_type_4 BHK_Independent Floor,building_type_4 BHK_Villa,building_type_5 BHK_Independent Floor,building_type_5 BHK_Villa,building_type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.106937,-0.086893,1.223499,1.020101,-0.181158,1.044465,0.553787,0.370864,...,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.106937,-0.086893,-0.893541,-0.896660,-0.291166,-0.458823,0.293086,0.472749,...,-0.28931,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [29]:
ohe_df_normalized.iloc[[0]].index[0]

'Smartworld One DXP'

In [30]:
#lets apply cosine simillarity
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_2 = cosine_similarity(ohe_df_normalized)
cosine_sim_2.shape

(246, 246)

In [31]:
sorted(list(enumerate(cosine_sim_2[0])), key = lambda x : x[1], reverse = True)

[(0, 1.0000000000000004),
 (79, 0.9744971479308603),
 (73, 0.9612396936057416),
 (3, 0.9605628102042169),
 (16, 0.9543685413356082),
 (91, 0.953512230863159),
 (80, 0.9448249550572729),
 (77, 0.9404479710856419),
 (22, 0.9368545562936695),
 (29, 0.9288672903930806),
 (234, 0.9228025220666316),
 (233, 0.9197020137070262),
 (118, 0.9021703365822388),
 (127, 0.883734779086525),
 (130, 0.8561551752625169),
 (124, 0.8437638378193391),
 (186, 0.835578430438821),
 (167, 0.8350827165862746),
 (219, 0.8258357412253282),
 (237, 0.8186806881733577),
 (137, 0.8084614677814299),
 (158, 0.798577858945976),
 (170, 0.7892257594651988),
 (99, 0.7776399127712843),
 (196, 0.7527333588308468),
 (192, 0.7510730253653046),
 (112, 0.7421929649463422),
 (9, 0.7156185509861334),
 (241, 0.6635632016909507),
 (126, 0.6545217013105072),
 (24, 0.6536176426090581),
 (30, 0.6271464414308986),
 (90, 0.626477147673952),
 (157, 0.6141001575614137),
 (133, 0.591825179378248),
 (27, 0.5751289812204059),
 (51, 0.558952295

In [32]:
def recommender_basedon_pricedetails(property_name):
  #get th index of the property
  idx = list(ohe_df_normalized.index).index(property_name)

  #get the cosine simillarity
  sim_scores = sorted(list(enumerate(cosine_sim_2[idx])), key = lambda x : x[1], reverse = True)

  #get top 6 simillar properties
  top_6 = sim_scores[:6]

  #get name and score of top 6 property
  simillar_properties = [ohe_df_normalized.iloc[[id]].index[0] for id, score in top_6]
  simillarity_score = [score for idx, score in top_6]
  pdet = [df[df['PropertyName'] == ohe_df_normalized.iloc[[id]].index[0]]['PriceDetails'].values for id, score in top_6]

  fdf = pd.DataFrame(
      {
          'property__name' : simillar_properties,
          'simillarity_score' : simillarity_score,
          'pricedetails' : pdet
      }
  )

  return fdf


In [33]:
recommender_basedon_pricedetails('DLF The Primus')

,property__name,simillarity_score,pricedetails
0,DLF The Primus,1.000000,"[{'3 BHK': {'building_type': 'Apartment', 'are..."
1,Bestech Altura,0.991763,"[{'3 BHK': {'building_type': 'Apartment', 'are..."
2,Conscient Heritage Max,0.991493,"[{'3 BHK': {'building_type': 'Apartment', 'are..."
3,DLF The Ultima,0.989199,"[{'3 BHK': {'building_type': 'Apartment', 'are..."
4,Bestech Park View Sanskruti,0.985915,"[{'3 BHK': {'building_type': 'Apartment', 'are..."
5,Orris Aster Court Premier,0.978539,"[{'3 BHK': {'building_type': 'Apartment', 'are..."


In [34]:
df['LocationAdvantages'] = df['LocationAdvantages'].str.lower().str.replace('meter', 'm').str.replace('mtrs', 'm')
df['LocationAdvantages']

0      {'bajghera road': '800 m', 'palam vihar halt':...
1      {'dpsg palam vihar gurugram': '1.4 km', 'the n...
2      {'aipl business club sector 62': '2.7 km', 'he...
3      {'the shikshiyan school': '2.9 km', 'wtc plaza...
4      {'pranavananda int. school': '450 m', 'dlf sit...
                             ...                        
242    {'sector 42-43 metro station': '1.8 km', 'para...
243    {'aarvy healthcare super speciality': '1.8 km'...
244    {'dwarka expressway': '1.2 km', 's n internati...
245    {'suncity school': '0.2 km', 'gurugram road': ...
246    {'sector 84 road': '600 m', 'delhi public scho...
Name: LocationAdvantages, Length: 246, dtype: object

In [35]:
def distance_to_meters(distance_str):
  try:
    if 'km' in distance_str:
        return float(distance_str.replace('km', '').split(' ')[0]) * 1000
    else :
        return float(distance_str.split()[0].replace('m',''))
  except:
    if 'close' in distance_str or 'in' in distance_str:
        return 0


In [36]:
# Extract distances for each location
location_matrix = {}
for index, row in df.iterrows():
    distances = {}
    for location, distance in ast.literal_eval(row['LocationAdvantages']).items():
        distances[location] = distance_to_meters(distance)
    location_matrix[row['PropertyName']] = distances

# Convert the dictionary to a dataframe
location_df = pd.DataFrame.from_dict(location_matrix, orient='index')

# Display the first few rows
location_df.shape

(246, 1026)

In [45]:
location_df.head()

,bajghera road,palam vihar halt,dpsg palam vihar,park hospital,gurgaon railway station,the northcap university,dwarka expy,hyatt place gurgaon udyog vihar,"dwarka sector 21, metro station",pacific d21 mall,...,mcc cricket ground dhankot,the shri ram school aravali,taj city centre gurugram,minda industries corporate office,"rampura flyover, naurangpur rd",manesar toll plaza - kherki daula,"imt manesar, gurugram",holiday inn,sector 84 road,skyview corporate park
Smartworld One DXP,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
La Vida by Tata Housing,550.0,5400.0,5400.0,5400.0,5400.0,6700.0,3800.0,5400.0,5400.0,7500.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Oxirich Chintamanis,5300.0,5400.0,5400.0,5400.0,2500.0,8800.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Rishali Luxe Residency 112,1500.0,5400.0,5400.0,5400.0,6500.0,6700.0,5100.0,5400.0,5400.0,8200.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Emaar Digihomes,5400.0,5400.0,5400.0,5500.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0


In [50]:
import pickle

with open('location_df.pkl', 'wb') as f:
  pickle.dump(location_df, f)

In [49]:
location_df

,bajghera road,palam vihar halt,dpsg palam vihar,park hospital,gurgaon railway station,the northcap university,dwarka expy,hyatt place gurgaon udyog vihar,"dwarka sector 21, metro station",pacific d21 mall,...,mcc cricket ground dhankot,the shri ram school aravali,taj city centre gurugram,minda industries corporate office,"rampura flyover, naurangpur rd",manesar toll plaza - kherki daula,"imt manesar, gurugram",holiday inn,sector 84 road,skyview corporate park
Smartworld One DXP,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
La Vida by Tata Housing,550.0,5400.0,5400.0,5400.0,5400.0,6700.0,3800.0,5400.0,5400.0,7500.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Oxirich Chintamanis,5300.0,5400.0,5400.0,5400.0,2500.0,8800.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Rishali Luxe Residency 112,1500.0,5400.0,5400.0,5400.0,6500.0,6700.0,5100.0,5400.0,5400.0,8200.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Emaar Digihomes,5400.0,5400.0,5400.0,5500.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Close South,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
DLF New Town Heights 2,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
DLF New Town Heights 1,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0
Antriksh Heights,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,...,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0,5400.0


Now we will have to replace nan value.

Note that we can not replace nan with 0 because then ml model will think that particular location is nearet. since there will be lot of 0 after replacing nan with 0 hence ml model will predict same score for all so to avoid that we will replace nan with max value of distance i.e 54000

In [37]:
location_df.fillna(5400, inplace = True)

In [38]:
#standerdization

scale = StandardScaler()

scaled_location_df = pd.DataFrame(scale.fit_transform(location_df), columns = location_df.columns, index = location_df.index)

In [39]:
cosine_sim_3 = cosine_similarity(scaled_location_df)

In [40]:
cosine_sim_3.shape

(246, 246)

In [41]:
def recommender_basedon_pricedetails(property_name):
  #get th index of the property
  idx = list(scaled_location_df.index).index(property_name)

  #get the cosine simillarity
  sim_scores = sorted(list(enumerate(cosine_sim_3[idx])), key = lambda x : x[1], reverse = True)

  #get top 6 simillar properties
  top_6 = sim_scores[:6]

  #get name and score of top 6 property
  simillar_properties = [scaled_location_df.iloc[[id]].index[0] for id, score in top_6]
  simillarity_score = [score for idx, score in top_6]
  pdet = [df[df['PropertyName'] == scaled_location_df.iloc[[id]].index[0]]['LocationAdvantages'].values for id, score in top_6]

  fdf = pd.DataFrame(
      {
          'property__name' : simillar_properties,
          'simillarity_score' : simillarity_score,
          'LocationAdvantages' : pdet
      }
  )

  return fdf


In [42]:
recommender_basedon_pricedetails('Emaar Digihomes')

,property__name,simillarity_score,LocationAdvantages
0,Emaar Digihomes,1.000000,"[{'imperia mindspace': '140 m', 'aipl business..."
1,Emaar MGF Palm Terraces,0.078481,"[{'heritage school': '2 km', 'isbm college': '..."
2,Ireo City Plots,0.043745,"[{'yonex badminton stadium': '3 mins', 'cbr cr..."
3,Vatika Express City,0.011405,"[{'nh 352w': '4.9 km', 'euro international sch..."
4,Shree Vardhman City,0.009038,"[{'gd goenka world school': '600 m', 'old sohn..."
5,Godrej Air,0.007968,"[{'sector 86 road': '3 km', 'dwarka expy': '3...."


In [43]:
def recommend_properties_with_score(prop_name):

  cos_sim = (3*cosine_sim_1+ 20*cosine_sim_2 + cosine_sim_3)
  idx = df[df['PropertyName'] == prop_name].index[0]

  sim_score = sorted(list(enumerate(cos_sim[idx])), key = lambda x : x[1], reverse = True)

  prop_name = [df['PropertyName'].iloc[[indx]].values[0] for indx, score in sim_score[:6]]
  score = [score for indx, score in sim_score[:6]]

  recommendations_df = pd.DataFrame(
      {
          "PropertyName" : prop_name,
          "score" : score
      }
  )

  return recommendations_df

In [44]:
recommend_properties_with_score("Shree Vardhman City")

,PropertyName,score
0,Signature Global Prime,24.000000
1,Suncity Avenue 102,20.381513
2,Breez Global Heights 89,19.764464
3,Zara Rossa,19.403768
4,BPTP Pedestal,17.573593
5,M3M Natura,17.314735


In [51]:
with open('cosine_sim_1.pkl', 'wb') as file:
  pickle.dump(cosine_sim_1, file)

with open('cosine_sim_2.pkl', 'wb') as file:
  pickle.dump(cosine_sim_2, file)

with open('cosine_sim_3.pkl', 'wb') as file:
  pickle.dump(cosine_sim_3, file)

In [52]:
df

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'bajghera road': '800 m', 'palam vihar halt':...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Swimming Pool Salon Restaurant Spa Cafeteria S...
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'dpsg palam vihar gurugram': '1.4 km', 'the n...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...",Bowling Alley Mini Theatre Manicured Garden Sw...
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'aipl business club sector 62': '2.7 km', 'he...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,Terrace Garden Gazebo Fountain Amphitheatre Pa...
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'the shikshiyan school': '2.9 km', 'wtc plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Swimming Pool Volley Ball Court Aerobics Centr...
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'pranavananda int. school': '450 m', 'dlf sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,Mini Theatre Doctor on Call Concierge Service ...
...,...,...,...,...,...,...,...
242,DLF Princeton Estate,"2, 3, 4 BHK Apartment in DLF Phase 5, Gurgaon","['Sector 42-43 Metro Station', 'Paras Hospital...","{'sector 42-43 metro station': '1.8 km', 'para...",https://www.99acres.com/dlf-princeton-estate-d...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Swimming Pool Medical Centre Laundry Salon Gro...
243,Pyramid Urban Homes 2,"1, 2 BHK Apartment in Sector 86, Gurgaon","['Aarvy Healthcare Super Speciality', ""St. Xav...",{'aarvy healthcare super speciality': '1.8 km'...,https://www.99acres.com/pyramid-urban-homes-2-...,"{'1 BHK': {'building_type': 'Apartment', 'area...",Shopping Centre Community Hall 24x7 Security G...
244,Satya The Hermitage,"2, 3, 4, 5 BHK Apartment in Sector 103, Gurgaon","['Dwarka Expressway', 'S N International Schoo...","{'dwarka expressway': '1.2 km', 's n internati...",https://www.99acres.com/satya-the-hermitage-se...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Bus Shelter Swimming Pool Business Lounge Read...
245,BPTP Spacio,"2, 3 BHK Apartment in Sector 37D, Gurgaon","['Suncity School', 'Gurugram Road', 'Dwarka Ex...","{'suncity school': '0.2 km', 'gurugram road': ...",https://www.99acres.com/bptp-spacio-sector-37d...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Swimming Pool Card Room Piped Gas Pool Table S...
